### Process
- import dependencies
- load data
- data preprocessing
    - feature selection
    - pearson correlation
    - 
- test-train split
- do classification
    - bagging
    - boosting
    - stacking
    - traditional ml techniques
- tables comparing accuracies, precesion, recall, f1
- different graphs

In [89]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [90]:
clinical_data = pd.read_csv('data/recent-data.csv', index_col=False)

In [91]:
clinical_data['Case ID'].str.startswith('AMC').sum()

49

In [92]:
clinical_data.head(2)

,Case ID,Patient affiliation,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,%GG,...,Recurrence,Recurrence Location,Date of Recurrence,Date of Last Known Alive,Survival Status,Date of Death,Time to Death (days),CT Date,Days between CT and surgery,PET Date
0,AMC-001,Stanford,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,yes,distant,10/7/1994,1/7/1997,Dead,1/7/1997,872.0,8/10/1994,9,Not Collected
1,AMC-002,Stanford,33,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,Not Assessed,...,no,NaN,NaN,3/20/1992,Alive,NaN,NaN,2/19/1992,3,Not Collected


## Data Pre-processing

In [93]:
clinical_data.columns

Index(['Case ID', 'Patient affiliation', 'Age at Histological Diagnosis',
       'Weight (lbs)', 'Gender', 'Ethnicity', 'Smoking status', 'Pack Years',
       'Quit Smoking Year', '%GG', 'Tumor Location (choice=RUL)',
       'Tumor Location (choice=RML)', 'Tumor Location (choice=RLL)',
       'Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)',
       'Tumor Location (choice=L Lingula)', 'Tumor Location (choice=Unknown)',
       'Histology ', 'Pathological T stage', 'Pathological N stage',
       'Pathological M stage', 'Histopathological Grade',
       'Lymphovascular invasion',
       'Pleural invasion (elastic, visceral, or parietal)',
       'EGFR mutation status', 'KRAS mutation status',
       'ALK translocation status', 'Adjuvant Treatment', 'Chemotherapy',
       'Radiation', 'Recurrence', 'Recurrence Location', 'Date of Recurrence',
       'Date of Last Known Alive', 'Survival Status', 'Date of Death',
       'Time to Death (days)', 'CT Date', 'Days between CT and sur

In [94]:
#remove caseid and patient affiliation
clinical_data.drop(columns=['Case ID', 'Patient affiliation'], inplace=True)


In [95]:
clinical_data.iloc[48]['Recurrence']

'Not collected'

In [96]:
clinical_data.drop(index=[48], inplace=True)

In [97]:
clinical_data.iloc[48]['Recurrence']

'no'

In [98]:
clinical_data.columns

Index(['Age at Histological Diagnosis', 'Weight (lbs)', 'Gender', 'Ethnicity',
       'Smoking status', 'Pack Years', 'Quit Smoking Year', '%GG',
       'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
       'Tumor Location (choice=RLL)', 'Tumor Location (choice=LUL)',
       'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)',
       'Tumor Location (choice=Unknown)', 'Histology ', 'Pathological T stage',
       'Pathological N stage', 'Pathological M stage',
       'Histopathological Grade', 'Lymphovascular invasion',
       'Pleural invasion (elastic, visceral, or parietal)',
       'EGFR mutation status', 'KRAS mutation status',
       'ALK translocation status', 'Adjuvant Treatment', 'Chemotherapy',
       'Radiation', 'Recurrence', 'Recurrence Location', 'Date of Recurrence',
       'Date of Last Known Alive', 'Survival Status', 'Date of Death',
       'Time to Death (days)', 'CT Date', 'Days between CT and surgery',
       'PET Date'],
      dtype='o

In [99]:
clinical_data['Weight (lbs)'].describe

<bound method NDFrame.describe of 0      Not Collected
1      Not Collected
2      Not Collected
3      Not Collected
4      Not Collected
           ...      
206              184
207            231.5
208    Not Collected
209              158
210              229
Name: Weight (lbs), Length: 210, dtype: object>

In [100]:
#repacing not collected with nan and calculating medeian to replace nan
clinical_data['Weight (lbs)'].replace('Not Collected', 'NaN', inplace=True)
clinical_data['Weight (lbs)'].replace('NaN', np.nan, inplace=True)

In [101]:
#replace Not Collected with NaN
clinical_data.replace('Not Collected', 'NaN', inplace=True)
clinical_data.replace('Not collected', 'NaN', inplace=True)
clinical_data.replace('Not Recorded In Database', 'NaN', inplace=True)
clinical_data.replace('NaN', np.nan, inplace=True)

In [102]:
clinical_data['Weight (lbs)'].describe()

count         152
unique        101
top       200.655
freq            7
Name: Weight (lbs), dtype: object

In [103]:
median_value = clinical_data['Weight (lbs)'].median()

In [104]:
clinical_data['Weight (lbs)'].replace(np.nan, median_value, inplace=True)

In [105]:
clinical_data['Weight (lbs)'].describe()

count     210.0
unique    102.0
top       171.0
freq       58.0
Name: Weight (lbs), dtype: float64

In [106]:
clinical_data.columns

Index(['Age at Histological Diagnosis', 'Weight (lbs)', 'Gender', 'Ethnicity',
       'Smoking status', 'Pack Years', 'Quit Smoking Year', '%GG',
       'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
       'Tumor Location (choice=RLL)', 'Tumor Location (choice=LUL)',
       'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)',
       'Tumor Location (choice=Unknown)', 'Histology ', 'Pathological T stage',
       'Pathological N stage', 'Pathological M stage',
       'Histopathological Grade', 'Lymphovascular invasion',
       'Pleural invasion (elastic, visceral, or parietal)',
       'EGFR mutation status', 'KRAS mutation status',
       'ALK translocation status', 'Adjuvant Treatment', 'Chemotherapy',
       'Radiation', 'Recurrence', 'Recurrence Location', 'Date of Recurrence',
       'Date of Last Known Alive', 'Survival Status', 'Date of Death',
       'Time to Death (days)', 'CT Date', 'Days between CT and surgery',
       'PET Date'],
      dtype='o

In [107]:
gender = pd.Categorical(clinical_data['Gender'])
#remove old gender column
clinical_data.drop(columns=['Gender'])

#create new gender column
clinical_data['Gender'] = gender

In [108]:
clinical_data.Gender

0        Male
1      Female
2      Female
3      Female
4        Male
        ...  
206      Male
207      Male
208    Female
209      Male
210      Male
Name: Gender, Length: 210, dtype: category
Categories (2, object): ['Female', 'Male']

In [109]:
clinical_data.columns

Index(['Age at Histological Diagnosis', 'Weight (lbs)', 'Gender', 'Ethnicity',
       'Smoking status', 'Pack Years', 'Quit Smoking Year', '%GG',
       'Tumor Location (choice=RUL)', 'Tumor Location (choice=RML)',
       'Tumor Location (choice=RLL)', 'Tumor Location (choice=LUL)',
       'Tumor Location (choice=LLL)', 'Tumor Location (choice=L Lingula)',
       'Tumor Location (choice=Unknown)', 'Histology ', 'Pathological T stage',
       'Pathological N stage', 'Pathological M stage',
       'Histopathological Grade', 'Lymphovascular invasion',
       'Pleural invasion (elastic, visceral, or parietal)',
       'EGFR mutation status', 'KRAS mutation status',
       'ALK translocation status', 'Adjuvant Treatment', 'Chemotherapy',
       'Radiation', 'Recurrence', 'Recurrence Location', 'Date of Recurrence',
       'Date of Last Known Alive', 'Survival Status', 'Date of Death',
       'Time to Death (days)', 'CT Date', 'Days between CT and surgery',
       'PET Date'],
      dtype='o

In [110]:
smoking_status = pd.Categorical(clinical_data['Smoking status'])
#remove old smoking_status column
clinical_data.drop(columns=['Smoking status'])

#create new smoking_status column
clinical_data['Smoking status'] = smoking_status

In [111]:
gg = pd.Categorical(clinical_data['%GG'])
#remove old gg column
clinical_data.drop(columns=['%GG'])

#create new gg column
clinical_data['%GG'] = gg

In [112]:
from numpy import NaN


clinical_data['Tumor Location'] = NaN

## Functions

In [113]:
def To_Categorical(column_name):
    column = pd.Categorical(clinical_data[column_name])
    #remove old smoking_status column
    clinical_data.drop(columns=[column_name])

    #create new smoking_status column
    clinical_data[column_name] = column

In [114]:
# To_Categorical('Tumor Location (choice=RUL)')
# To_Categorical('Tumor Location (choice=RML)')
# To_Categorical('Tumor Location (choice=RLL)')
# To_Categorical('Tumor Location (choice=LUL)')
# To_Categorical('Tumor Location (choice=LLL)')
# To_Categorical('Tumor Location (choice=Unknown)')
# To_Categorical('Tumor Location (choice=L Lingula)')

In [115]:
clinical_data.replace({'Tumor Location (choice=L Lingula)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=RUL)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=RML)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=RLL)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=LUL)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=LLL)': {'Checked':1, 'Unchecked':0}}, inplace=True)
clinical_data.replace({'Tumor Location (choice=Unknown)': {'Checked':1, 'Unchecked':0}}, inplace=True)

In [116]:
clinical_data.drop(columns=['Tumor Location'], inplace=True)

In [117]:
clinical_data['Histology'] = clinical_data['Histology ']
clinical_data.drop(columns=['Histology '], inplace=True)

In [118]:
To_Categorical('Histology')

In [119]:
To_Categorical('Pathological T stage')
To_Categorical('Pathological M stage')
To_Categorical('Pathological N stage')

In [120]:
To_Categorical('Histopathological Grade')

In [121]:
To_Categorical('Lymphovascular invasion')

In [122]:
# clinical_data['Pleural invasion (elastic, visceral, or parietal)'].value_counts()
To_Categorical('Pleural invasion (elastic, visceral, or parietal)')

In [123]:
# clinical_data['EGFR mutation status'].value_counts()
To_Categorical('EGFR mutation status')

In [124]:
# clinical_data['KRAS mutation status'].value_counts()
To_Categorical('KRAS mutation status')

In [125]:
clinical_data['Adjuvant Treatment'].value_counts()
# To_Categorical('Adjuvant Treatment')
clinical_data.replace({'Adjuvant Treatment': {'Yes':1, 'No':0}}, inplace=True)

In [126]:
clinical_data['Chemotherapy'].value_counts()
clinical_data.replace({'Chemotherapy': {'Yes':1, 'No':0}}, inplace=True)

In [127]:
clinical_data['Radiation'].value_counts()
clinical_data.replace({'Radiation': {'Yes':1, 'No':0}}, inplace=True)

In [128]:
clinical_data['Recurrence'].value_counts()
clinical_data.replace({'Recurrence': {'yes':1, 'no':0}}, inplace=True)
# To_Categorical('Recurrence')

In [129]:

# clinical_data['Recurrence Location'].value_counts()
To_Categorical('Recurrence Location')

In [130]:
# remove columns 'Date of Recurrence','Date of Last Known Alive', 'Survival Status', 'Date of Death',

clinical_data.drop(columns=['Date of Recurrence','Date of Last Known Alive', 'Survival Status', 'Date of Death', 'Time to Death (days)', 'Recurrence Location'], inplace=True)
clinical_data.drop(columns=['PET Date', 'CT Date'], inplace=True)
clinical_data.drop(columns=['Quit Smoking Year'], inplace=True)

In [132]:

clinical_data['Histology'].value_counts()
# To_Categorical('Histology')

Adenocarcinoma                         171
Squamous cell carcinoma                 35
NSCLC NOS (not otherwise specified)      4
Name: Histology, dtype: int64

In [133]:
clinical_data.columns

Index(['Age at Histological Diagnosis', 'Weight (lbs)', 'Gender', 'Ethnicity',
       'Smoking status', 'Pack Years', '%GG', 'Tumor Location (choice=RUL)',
       'Tumor Location (choice=RML)', 'Tumor Location (choice=RLL)',
       'Tumor Location (choice=LUL)', 'Tumor Location (choice=LLL)',
       'Tumor Location (choice=L Lingula)', 'Tumor Location (choice=Unknown)',
       'Pathological T stage', 'Pathological N stage', 'Pathological M stage',
       'Histopathological Grade', 'Lymphovascular invasion',
       'Pleural invasion (elastic, visceral, or parietal)',
       'EGFR mutation status', 'KRAS mutation status',
       'ALK translocation status', 'Adjuvant Treatment', 'Chemotherapy',
       'Radiation', 'Recurrence', 'Days between CT and surgery', 'Histology'],
      dtype='object')

In [134]:
# # ethnicity 49 not recorded
# clinical_data.Ethnicity.vaalue_counts()['Not Recorded In Database']

# #pack years - total 163
# clinical_data['Pack Years'].count()

# #Quit Smoking years - total 108
# clinical_data['Quit Smoking Year'].count()



In [135]:
clinical_data

,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,%GG,Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),...,"Pleural invasion (elastic, visceral, or parietal)",EGFR mutation status,KRAS mutation status,ALK translocation status,Adjuvant Treatment,Chemotherapy,Radiation,Recurrence,Days between CT and surgery,Histology
0,34,171.0,Male,NaN,Nonsmoker,NaN,Not Assessed,0,0,1,...,NaN,Wildtype,Mutant,Wildtype,0,0,0,1,9,Adenocarcinoma
1,33,171.0,Female,NaN,Nonsmoker,NaN,Not Assessed,0,0,0,...,NaN,Wildtype,Wildtype,NaN,0,0,0,0,3,Adenocarcinoma
2,69,171.0,Female,NaN,Nonsmoker,NaN,Not Assessed,0,0,1,...,NaN,Mutant,Wildtype,Wildtype,0,0,0,0,28,Adenocarcinoma
3,80,171.0,Female,NaN,Nonsmoker,NaN,Not Assessed,0,0,0,...,NaN,Wildtype,Wildtype,Wildtype,0,0,0,0,47,Adenocarcinoma
4,76,171.0,Male,NaN,Former,30,Not Assessed,0,0,0,...,NaN,Mutant,Wildtype,Wildtype,0,0,0,1,2,Adenocarcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,75,184,Male,Caucasian,Former,55,Not Assessed,0,0,0,...,No,Unknown,Unknown,Unknown,0,0,0,0,14,Squamous cell carcinoma
207,61,231.5,Male,Caucasian,Former,12,Not Assessed,1,0,0,...,No,Unknown,Mutant,Unknown,1,1,0,0,72,Adenocarcinoma
208,52,171.0,Female,Caucasian,Former,7,Not Assessed,0,0,0,...,No,Wildtype,Wildtype,Wildtype,0,0,0,0,8,Adenocarcinoma
209,67,158,Male,Asian,Former,15,Not Assessed,0,0,0,...,No,Mutant,Wildtype,Wildtype,0,0,0,0,65,Adenocarcinoma


In [136]:
# clinical_data['Weight (lbs)']

In [137]:
#weight to numerical value
clinical_data['Weight (lbs)'] = pd.to_numeric(clinical_data['Weight (lbs)'])
# clinical_data.astype({'Weight (lbs)':'Int32'})

In [138]:
# clinical_data['Quit Smoking'] = clinical_data['Quit Smoking Year'].isnull()
# clinical_data['Quit Smoking Year'] = clinical_data.drop('Quit Smoking Year', axis=1)

In [139]:
clinical_data['Weight (lbs)']

0      171.0
1      171.0
2      171.0
3      171.0
4      171.0
       ...  
206    184.0
207    231.5
208    171.0
209    158.0
210    229.0
Name: Weight (lbs), Length: 210, dtype: float64

In [140]:
clinical_data['Recurrence'].value_counts()

0    156
1     54
Name: Recurrence, dtype: int64

In [141]:
156+54

210

Train Test Split

In [142]:
print(156/210)
print(54/210)

0.7428571428571429
0.2571428571428571


In [143]:
0.8*210

168.0

In [144]:
0.25 * 168

42.0

In [145]:
clinical_data['Recurrence'].value_counts()

0    156
1     54
Name: Recurrence, dtype: int64

In [146]:
# 42 no data from recurrence
# 168-42 = 126 yes data from recurrence

In [147]:
# get only reccurrence yes values
clinical_data['Recurrence'][clinical_data['Recurrence'] == 1]

0      1
4      1
10     1
11     1
33     1
37     1
38     1
43     1
52     1
53     1
55     1
58     1
60     1
64     1
66     1
68     1
70     1
72     1
74     1
77     1
78     1
79     1
81     1
83     1
84     1
98     1
99     1
102    1
113    1
116    1
120    1
123    1
125    1
136    1
139    1
141    1
146    1
147    1
151    1
157    1
159    1
161    1
167    1
170    1
173    1
176    1
189    1
194    1
195    1
196    1
197    1
199    1
201    1
210    1
Name: Recurrence, dtype: int64

In [148]:
# get only reccurrence yes values
clinical_data['Recurrence'][clinical_data['Recurrence'] == 0]

1      0
2      0
3      0
5      0
6      0
      ..
205    0
206    0
207    0
208    0
209    0
Name: Recurrence, Length: 156, dtype: int64

In [149]:
X = clinical_data.copy()
X.drop(columns=['Recurrence'], inplace=True)
Y = clinical_data['Recurrence']

## Need to create a mechanism to train test by myself

In [150]:
from sklearn.preprocessing import OneHotEncoder

In [151]:
X['Gender']

0        Male
1      Female
2      Female
3      Female
4        Male
        ...  
206      Male
207      Male
208    Female
209      Male
210      Male
Name: Gender, Length: 210, dtype: category
Categories (2, object): ['Female', 'Male']

In [152]:
one_hot_encoded_data = pd.get_dummies(X)

In [153]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(one_hot_encoded_data, Y, test_size = 0.30, random_state = 0)

In [154]:
clinical_data['Tumor Location (choice=RUL)']

0      0
1      0
2      0
3      0
4      0
      ..
206    0
207    1
208    0
209    0
210    0
Name: Tumor Location (choice=RUL), Length: 210, dtype: int64

Feature Scaling

In [155]:
clinical_data.describe()

,Age at Histological Diagnosis,Weight (lbs),Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),Tumor Location (choice=LUL),Tumor Location (choice=LLL),Tumor Location (choice=L Lingula),Tumor Location (choice=Unknown),Adjuvant Treatment,Chemotherapy,Radiation,Recurrence,Days between CT and surgery
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.00000,210.0,210.000000,210.000000,210.000000,210.000000,210.000000
mean,67.928571,169.904878,0.366667,0.085714,0.142857,0.223810,0.157143,0.02381,0.0,0.233333,0.233333,0.076190,0.257143,50.352381
std,10.017671,35.618808,0.483046,0.280611,0.350763,0.417792,0.364805,0.15282,0.0,0.423963,0.423963,0.265937,0.438103,62.486053
min,24.000000,49.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
25%,64.000000,152.145000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,20.250000
50%,68.000000,171.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,38.500000
75%,75.000000,184.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,1.000000,65.750000
max,87.000000,317.520000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.0,1.000000,1.000000,1.000000,1.000000,735.000000


In [156]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [157]:
np.array(y_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [158]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [172]:
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

from sklearn.svm import SVC
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)


from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

from sklearn.ensemble import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train, y_train)


GradientBoostingClassifier()

In [173]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score, precision_score
def predict(classifier):
    y_predict = classifier.predict(X_test)

    return y_predict

def print_evaluations(y_predict):
    cm = confusion_matrix(y_test, y_predict)
    ac = accuracy_score(y_test,y_predict)
    rs = recall_score(y_test,y_predict)
    ps = precision_score(y_test,y_predict)

    print(cm)
    print(ac)
    print(rs)
    print(ps)


In [174]:
nb_y_predict = predict(nb_classifier)
print('\nnb_y_predict')
print_evaluations(nb_y_predict)

svm_y_predict = predict(svm_classifier)
print('\nsvm_y_predict')
print_evaluations(svm_y_predict)

dt_y_predict = predict(dt_classifier)
print('\ndt_y_predict')
print_evaluations(dt_y_predict)

gb_y_predict = predict(gb_classifier)
print('\ngb_y_predict')
print_evaluations(gb_y_predict)


nb_y_predict
[[15 27]
 [ 7 14]]
0.4603174603174603
0.6666666666666666
0.34146341463414637

svm_y_predict
[[42  0]
 [21  0]]
0.6666666666666666
0.0
0.0

dt_y_predict
[[39  3]
 [16  5]]
0.6984126984126984
0.23809523809523808
0.625

gb_y_predict
[[40  2]
 [18  3]]
0.6825396825396826
0.14285714285714285
0.6


/Users/ms/miniforge3/envs/tensorflow-metal/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


k-fold validation

In [162]:
from sklearn.model_selection import cross_val_score

In [163]:
cross_val_score(GaussianNB(), one_hot_encoded_data, Y)

array([0.26190476, 0.33333333, 0.45238095, 0.35714286, 0.4047619 ])

In [164]:
cross_val_score(SVC(), one_hot_encoded_data, Y)

array([0.73809524, 0.73809524, 0.73809524, 0.73809524, 0.76190476])

In [165]:
cross_val_score(DecisionTreeClassifier(), one_hot_encoded_data, Y)

array([0.71428571, 0.78571429, 0.5952381 , 0.61904762, 0.66666667])

In [166]:
cross_val_score(GradientBoostingClassifier(), one_hot_encoded_data, Y)

array([0.73809524, 0.80952381, 0.64285714, 0.64285714, 0.78571429])

In [175]:
from sklearn.ensemble import AdaBoostClassifier
adaboost_classifier = AdaBoostClassifier()
adaboost_classifier.fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)


RandomForestClassifier()

In [177]:
adaboost_y_predict = predict(adaboost_classifier)
print('\nadabost_y_predict')
print_evaluations(adaboost_y_predict)

rf_y_predict = predict(rf_classifier)
print('\nrf_y_predict')
print_evaluations(rf_y_predict)


adabost_y_predict
[[34  8]
 [16  5]]
0.6190476190476191
0.23809523809523808
0.38461538461538464

rf_y_predict
[[40  2]
 [19  2]]
0.6666666666666666
0.09523809523809523
0.5


In [194]:
from sklearn.ensemble import BaggingClassifier
bag_model = BaggingClassifier(
base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=1.0, 
    bootstrap=True,
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  oob_score=True, random_state=0)

In [195]:
bag_y_predict = predict(bag_model)
print('\nrf_y_predict')
print_evaluations(bag_y_predict)


rf_y_predict
[[39  3]
 [17  4]]
0.6825396825396826
0.19047619047619047
0.5714285714285714


In [196]:
cross_val_score(bag_model, one_hot_encoded_data, Y)

array([0.76190476, 0.78571429, 0.64285714, 0.69047619, 0.76190476])